# Configuration

RadiObject's write, read, and S3 settings. For the full parameter reference, see [configuration.md](https://srdsam.github.io/RadiObject/reference/configuration/).

**Prerequisites:** [01_explore_data.ipynb](./01_explore_data.ipynb)

In [1]:
import shutil
import tempfile
from pathlib import Path

import numpy as np

from radiobject import (
    CompressionConfig,
    Compressor,
    ReadConfig,
    S3Config,
    SliceOrientation,
    TileConfig,
    Volume,
    WriteConfig,
    configure,
    get_radiobject_config,
    reset_radiobject_config,
)

TEMP_DIR = tempfile.mkdtemp(prefix="config_tutorial_")

## Write Settings

Write settings (tile layout, compression) are **immutable after ingestion**.

In [2]:
configure(
    write=WriteConfig(
        tile=TileConfig(orientation=SliceOrientation.AXIAL),
        compression=CompressionConfig(algorithm=Compressor.ZSTD, level=3),
    )
)

config = get_radiobject_config()
print(f"Tile orientation: {config.write.tile.orientation}")
print(f"Compression: {config.write.compression.algorithm}, level={config.write.compression.level}")

Tile orientation: SliceOrientation.AXIAL
Compression: Compressor.ZSTD, level=3


In [3]:
# Tile extents for each orientation
shape = (240, 240, 155)

for orient in SliceOrientation:
    extents = TileConfig(orientation=orient).extents_for_shape(shape)
    print(f"{orient.value:10s} -> tile extents: {extents}")

axial      -> tile extents: (240, 240, 1)
sagittal   -> tile extents: (1, 240, 155)
coronal    -> tile extents: (240, 1, 155)
isotropic  -> tile extents: (64, 64, 64)


In [4]:
# Create a volume with current write settings
test_data = np.random.randn(64, 64, 32).astype(np.float32)
uri = str(Path(TEMP_DIR) / "vol_axial")
vol = Volume.from_numpy(uri, test_data)
print(f"Created: {vol}")

Created: Volume(shape=64x64x32, dtype=float32)


## Read and S3 Settings

Read settings affect all data access. S3 settings configure cloud storage.

In [5]:
configure(
    read=ReadConfig(
        memory_budget_mb=2048,
        concurrency=8,
        max_workers=4,
    )
)

config = get_radiobject_config()
print(f"Memory budget: {config.read.memory_budget_mb} MB")
print(f"I/O concurrency: {config.read.concurrency} threads")
print(f"Max workers: {config.read.max_workers}")

Memory budget: 2048 MB
I/O concurrency: 8 threads
Max workers: 4


In [6]:
configure(
    s3=S3Config(
        region="us-east-2",
        max_parallel_ops=16,
        multipart_part_size_mb=50,
    )
)

config = get_radiobject_config()
print(f"Region: {config.s3.region}")
print(f"Max parallel ops: {config.s3.max_parallel_ops}")
print(f"Multipart size: {config.s3.multipart_part_size_mb} MB")

Region: us-east-2
Max parallel ops: 16
Multipart size: 50 MB


## Inspect and Reset

In [7]:
config = get_radiobject_config()

print("=== Write ===")
print(f"  Tile: {config.write.tile.orientation.value}")
print(
    f"  Compression: {config.write.compression.algorithm.value} (level {config.write.compression.level})"
)

print("\n=== Read ===")
print(f"  Memory budget: {config.read.memory_budget_mb} MB")
print(f"  Concurrency: {config.read.concurrency} threads")

print("\n=== S3 ===")
print(f"  Region: {config.s3.region}")
print(f"  Max parallel ops: {config.s3.max_parallel_ops}")

=== Write ===
  Tile: axial
  Compression: zstd (level 3)

=== Read ===
  Memory budget: 2048 MB
  Concurrency: 8 threads

=== S3 ===
  Region: us-east-2
  Max parallel ops: 16


In [8]:
print(
    f"Before reset: tile={get_radiobject_config().write.tile.orientation.value}, memory={get_radiobject_config().read.memory_budget_mb}MB"
)

reset_radiobject_config()

print(
    f"After reset:  tile={get_radiobject_config().write.tile.orientation.value}, memory={get_radiobject_config().read.memory_budget_mb}MB"
)

Before reset: tile=axial, memory=2048MB
After reset:  tile=axial, memory=1024MB


In [9]:
shutil.rmtree(TEMP_DIR)

## Next Steps

- [Configuration Reference](https://srdsam.github.io/RadiObject/reference/configuration/) - Full parameter reference
- [Benchmarks](https://srdsam.github.io/RadiObject/reference/benchmarks/) - Tiling and compression benchmarks
- [03_ingest_msd.ipynb](./03_ingest_msd.ipynb) - Multi-collection ingestion
- [04_ml_training.ipynb](./04_ml_training.ipynb) - MONAI DataLoader integration